In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as skl

# 数据预处理

将match_id重新搞成易读取的形式

将时间戳化为秒

In [38]:
data = pd.read_csv('Wimbledon_featured_matches.csv')

# 修改时间戳
def time_trans(time):
    hours, minutes, seconds = map(int, time.split(':'))
    return hours*3600+minutes*60+seconds

data['elapsed_time'] = data['elapsed_time'].apply(time_trans)

# 修改id
idDict = {}
for id in data['match_id'].unique():
    if id not in idDict:
        idDict[id] = len(idDict)
data['match_id'] = data['match_id'].map(idDict)

# 网球比赛的比分

## 一局

（1）每胜1球得1分，先胜4分者胜1局。
（2）双方各得3分时为“平分”，平分后，净胜两分为胜1局。国际网球赛都是用英语单词Love，以及数字15、30、40分别表示0、1、3、4。

## 一盘

（1）一方先胜6局为胜1盘。
（2）双方各胜5局时，一方净胜两局为胜1盘

### 决胜局计分制

在每盘的局数为6平时，有以下两种计分制。
（1）长盘制：一方净胜两局为胜1盘。
（2）短盘制（抢七）：决胜盘除外，除非赛前另有规定，一般应按以下办法执行。

A.先得7分者为胜该局及该盘（若分数为6平时，一方须净两分）。
B.首先发球员发第1分球，对方发第2、3分球，然后轮流发两分球，直到比赛 结束。
C.第1分球在右区发，第2分球在左区发，第3分球在右区发。
D.每6分球和决胜局结束都要交换场地。

短盘制的计分
（1）第1个球（0：0），发球员A发1分球，1分球之后换发球。
（2）第2、3个球（报1：0或0：1，不报15：0或0：15），由B发球，B连发两分球后换发球，先从左区发球。
（3）第4、5个球（报3：0或1：2，2：1，不报40：0或15：30， 30：15），由A发球，A连发两球后换发球后换发球，先从左区发球。
（4）第6、7个球（报3：3或2：4，4：2或1：5，5：1或6：0，0： 6） ，由B发1分球之后交换场地，若比赛未结束，B继续发第7个球。
（5）比分打到5：5，6：6，7：7，8：8……时，需连胜两分才能决定谁为胜方。但在记分表上则统一写为7：6。
（6）决胜局打完之后，双方队员交换场地。

五盘三胜制度

# 题目1

1. 给出整个比赛流程中哪一个更好的模型（以及好了多少），并对比赛流程进行可视化。

# 思路

特定时段内表现更好，使用特定时间段内的得分数来量化。另外，可计算得分概率，发球方的得分概率有一定的修正。

可试图建立一个长期性的模型，在这个模型中，早期表现的影响会逐渐变小，以反映最近的表现好与坏。最终得到某一定的时间的pp分。可以通过前一局的情况得出。

将表现情况与发球情况进行分类，得到下一球的得分概率。

In [39]:
for i in data['match_id'].unique():
    matchData = data[data['match_id'] == i]
    pp = np.zeros(matchData.shape[0])
    # for t in range(0, matchData.shape(0)):